In [ ]:
#pip install numpy opencv-python matplotlib pandas tensorflow scikit-learn

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def set_train_data(train_data_dir, num_classes,
                   resize_col=32, resize_row=32,
                   test_size=0.2, random_state=1):

    
    data = []
    labels = []

    train_files = os.listdir(train_data_dir)
    for classes in train_files:

        classname = str(classes)
        path = os.path.join(train_data_dir, classname)
        images = os.listdir(path)

        
        for image in images:

            image_path = os.path.join(path, image)          
            image_array = cv2.imread(image_path)
            image_array = cv2.resize(image_array, (resize_row, resize_col))
            data.append(image_array)
            labels.append(classname)

    data = np.array(data)
    labels = np.array(labels)



    x_train, x_val, y_train, y_val = train_test_split(
        data, labels, test_size=test_size, random_state=random_state)
    y_train = to_categorical(y_train, num_classes)
    y_val = to_categorical(y_val, num_classes)

    return x_train, y_train, x_val, y_val

In [ ]:
def set_test_data(dataset_dir,
                  test_info_dir, num_classes,
                  resize_col=32, resize_row=32):

    
    y_test = pd.read_csv(test_info_dir)
    
    images = y_test['filename'].values
    
    y_test = y_test["class"].values

    
    x_test = []

    
    for image in images:

        image_path = os.path.join(dataset_dir, image)
        
        image_array = cv2.imread(image_path)
        
        image_array = cv2.resize(image_array, (resize_row, resize_col))
        
        x_test.append(image_array)

    
    x_test = np.array(x_test)
    y_test = np.array(y_test)

    
    y_test = to_categorical(y_test, num_classes)

    return x_test, y_test

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.models import Sequential
import os

In [ ]:
def build_model(input_shape, num_classes,
                kernel_size=(3, 3),
                pool_size=(2, 2),
                dropout_rate=0.3):

    model = Sequential()

    model.add(Conv2D(32, kernel_size=kernel_size, padding='same',
                     input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(64, kernel_size=kernel_size, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(128, kernel_size=kernel_size, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(dropout_rate))

    model.add(Flatten())

    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout_rate))

    model.add(Dense(num_classes, activation="softmax"))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

    return model


In [ ]:
model = build_model((32,32,3),13)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        1

In [ ]:
def train_model(model,
                x_train, y_train,
                x_val, y_val,
                batch_size=16,
                epochs=10):
   
    history = model.fit(x_train, y_train,
                        batch_size=batch_size, epochs=epochs,
                        validation_data=(x_val, y_val))
    return model, history

In [ ]:
import matplotlib.pyplot as plt


def visualize_history(hist):

    
    histo = hist.history

    plt.style.use('default')
    plt.style.use('seaborn')

    plt.figure(0)
    plt.plot(histo['accuracy'], label='train accuracy')
    plt.plot(histo['val_accuracy'], label='val accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.savefig('train_accuracy')

    plt.figure(1)
    plt.plot(histo['loss'], label='train loss')
    plt.plot(histo['val_loss'], label='val loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.savefig('train_loss')


In [ ]:
# Verilerin hazırlanması
data_dir = "/content/drive/MyDrive/dataset/TrafficDataSet"
num_classes = 13
test_info_dir="/content/drive/MyDrive/dataset/TrafficDataSet/test.csv"
train_data_dir = os.path.join(data_dir, 'train')

x_train, y_train, x_val, y_val = set_train_data(
   train_data_dir, num_classes)

test_info_dir = os.path.join(data_dir, 'test.csv')
test_data_dir = os.path.join(data_dir, 'test')
x_test, y_test = set_test_data(test_data_dir, test_info_dir, num_classes)


In [ ]:
# Eğitim ve validasyon işlenmiş verilerinin kaydedilmesi
#processed_data_dir = '/content/drive/MyDrive/dataset/TrafficDataSet/processed_data/32'

#np.save(processed_data_dir+'/x_train.npy', x_train)
#np.save(processed_data_dir+'/y_train.npy', y_train)
#np.save(processed_data_dir+'/x_val.npy', x_val)
#np.save(processed_data_dir+'/y_val.npy', y_val)
#np.save(processed_data_dir+'/x_test.npy', x_val)
#np.save(processed_data_dir+'/y_test.npy', y_val)

In [ ]:
# Modelin eğitilmesi
input_shape = x_train.shape[1:]
model = build_model(input_shape, num_classes)
model, history = train_model(model, x_train, y_train, x_val, y_val)

Epoch 1/10
350/350 [==============================] - 23s 61ms/step - loss: 1.9960 - accuracy: 0.3469 - val_loss: 1.4191 - val_accuracy: 0.5282
Epoch 2/10
350/350 [==============================] - 21s 60ms/step - loss: 1.3262 - accuracy: 0.5519 - val_loss: 0.9419 - val_accuracy: 0.6948
Epoch 3/10
350/350 [==============================] - 23s 65ms/step - loss: 1.0207 - accuracy: 0.6679 - val_loss: 0.7227 - val_accuracy: 0.7956
Epoch 4/10
350/350 [==============================] - 21s 61ms/step - loss: 0.8305 - accuracy: 0.7291 - val_loss: 0.6082 - val_accuracy: 0.8385
Epoch 5/10
350/350 [==============================] - 22s 62ms/step - loss: 0.6849 - accuracy: 0.7743 - val_loss: 0.4423 - val_accuracy: 0.8785
Epoch 6/10
153/350 [============>.................] - ETA: 11s - loss: 0.6274 - accuracy: 0.7937

KeyboardInterrupt: ignored

In [ ]:
input_shape = x_train.shape[1:]
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
visualize_history(history)

In [ ]:
# Modelin kaydedilmesi
#model.save('/content/drive/MyDrive/dataset/TrafficDataSet/models/CNNmodel.h5')

In [ ]:
#np.save(processed_data_dir+'/CNNhistory.npy',history.history)

## **Modeli Test Etme**

In [ ]:
from tensorflow.keras.models import load_model



model_dir = '/content/drive/MyDrive/dataset/TrafficDataSet/models/CNNmodel.h5'

model = load_model(model_dir)

model.evaluate(x_test, y_test)

